## Scoring your model against a test set

This notebook shows you one way to score your model .pkl aginst a test set using the F1 Score metric. The test set is assumed to have the same file structure as your development set, e.g. Images located in ***testset/pos***, ***testset/neg***, with images in the ***pos*** directory considered labeled to have positive sentiments and the images in the ***neg*** directory labeled to have negative sentiments.

In [143]:
from fastbook import *
from fastai.vision.widgets import *
from fastai.callback.fp16 import *

In [144]:
path = Path('../models')
pickles=path.ls(file_exts='.pkl')

In [145]:
pickles

(#6) [Path('../models/model_resnet34_with8_negfocused.pkl'),Path('../models/model_resnet34_with32_negfocused.pkl'),Path('../models/model_resnet34_with32.pkl'),Path('../models/model_resnet34.pkl'),Path('../models/model_resnet34_with16_negfocused.pkl'),Path('../models/model_resnet34_64.pkl')]

In [146]:
learn_inf = load_learner(path/pickles[0])

In [147]:
learn_inf.dls.vocab

(#2) ['neg','pos']

In [148]:
df = pd.DataFrame()
fname_l=list()
pred_l=list()
prob_l=list()
target_l=list()

for lbl in learn_inf.dls.vocab:
    target=lbl
    path = Path('../testset/'+lbl)
    testimgs = path.ls(file_type='image/*')
#     pred,pred_idx,probs = learn_inf.predict(testimgs[0])
#     print(testimgs[0], ",", pred, ", %.6f"% probs[pred_idx])
    for img in testimgs:
        pred,pred_idx,probs = learn_inf.predict(img);
        _,_,_,fname=img.parts
        fname_l.append(fname)
        pred_l.append(pred)
        prob_l.append(probs[pred_idx].item())
        target_l.append(target)
#         print(fname, ",", pred, ", %.6f"% probs[pred_idx].item())


In [149]:
df = pd.DataFrame({'image_name':fname_l, 'pred':pred_l, 'prob':prob_l, 'target':target_l}, columns=['image_name','pred','prob','target'])

In [150]:
df.describe

<bound method NDFrame.describe of           image_name pred      prob target
0   neg_00000005.jpg  neg  0.988190    neg
1   neg_00000006.jpg  neg  0.998447    neg
2  neg_00000007.jpeg  neg  0.999457    neg
3   neg_00000004.jpg  neg  0.959892    neg
4  neg_00000002.jpeg  neg  0.984903    neg
5   pos_00000006.jpg  pos  0.999714    pos
6   pos_00000007.jpg  pos  0.995844    pos
7   pos_00000002.jpg  pos  0.997900    pos
8   pos_00000004.jpg  pos  0.999293    pos
9   pos_00000005.jpg  pos  0.999506    pos>

In [151]:
path = Path('../testset')
df.to_csv(path/'submission.csv', index=False)

In [152]:
path

Path('../testset')

In [153]:
from sklearn.metrics import f1_score

In [154]:
# Score for only entire test set

f1_score(df.target, df.pred, labels=learn_inf.dls.vocab, average='weighted')

1.0

In [155]:
# Score for only target=pos

f1_score(df[df['target'] == 'pos'].target, df[df['target'] == 'pos'].pred, labels=learn_inf.dls.vocab, average='weighted')

/anaconda/envs/fastai2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


1.0

In [156]:
# Score for only target=neg

f1_score(df[df['target'] == 'neg'].target, df[df['target'] == 'neg'].pred, labels=learn_inf.dls.vocab, average='weighted')

1.0